# 1 Load Data Set & Data Preprocessing
In this section the three data sets "website-phising.cvs", "bcp.csv" and "arrhythmia.csv" will be loaded using Pandas. When the data sets are loaded, there will also be some preprocessing before implementation of the classifing methods. ? will be replaces with NA, so that these values can be replaced by the mode of the column. The mode of the class is the majority value in the column. To make sure the categorical values can be used, they will be converted into numerical representation. This is due to errors in implementation.

In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


#Preprocessing of files
def read_file(filename):
    df = pd.read_csv(filename) #Read file
    df.replace('?', pd.NA, inplace=True) #Replace ? with NA
    df = df.drop(df.index[0]) #Drop header row
    replace_missing_values(df) #Replace the missing values
    numericalConverter(df)
    return df


#Convert categorical columns to numerical representation
def numericalConverter(df):
    df = df.astype(float)
    categorical_columns = df.select_dtypes(include=['object']).columns
    for col in categorical_columns:
        label_encoder = LabelEncoder()
        df[col] = label_encoder.fit_transform(df[col]) 

#Replace missing values with mode
def replace_missing_values(df):
    for column in df.columns:
        df[column] = df[column].fillna(df[column].mode()[0])

wp = read_file('website-phishing.csv')
bcp = read_file('bcp.csv')
ar = read_file('arrhythmia.csv')

#Printing preview of data sets
print("Website Phishing: ")
print(wp.head())
print("\nBCP: ")
print(bcp.head())
print("\nArrhythmia: ")
print(ar.head())


Website Phishing: 
   having_IP_Address   URL_Length   Shortining_Service   having_At_Symbol  \
1                  1            1                    1                  1   
2                  1            0                    1                  1   
3                  1            0                    1                  1   
4                  1            0                   -1                  1   
5                 -1            0                   -1                  1   

    double_slash_redirecting   Prefix_Suffix   having_Sub_Domain  \
1                          1              -1                   0   
2                          1              -1                  -1   
3                          1              -1                  -1   
4                          1              -1                   1   
5                         -1              -1                   1   

    SSLfinal_State   Domain_registeration_length   Favicon  ...  \
1                1                        

# 2 Implement Decision Trees
In this section the decision stump-, unpruned decision tree-, and pruned decision tree-methods will be implemented. After that the methods will be applied to the data sets.

## 2.1 Implementations of methods
### 2.1.1 Decision Stump
A decision stump is a decision tree with only one split. Since there is only one split, the possible splits will be evaluated based on accuracy, since that is what the model it self will be evaluated by. A problem that might arise is that we can see from section 1 that the data sets have more than to classes, and the method that will bli implemented only splits the data into two subsets. Therefore, it is expected to observe a low accuracy when implementing this model in section 2.2. 

In [36]:
import numpy as np

class DecisionStump:
    def __init__(self):
        self.feature_index = None #Split feature
        self.threshold = None #Split value
        self.classes = None #class labels

    def fit(self, X, y):
        self.classes = np.unique(y) #get class labels
        best_accuracy = 0

        # Iterate over each feature and threshold to find the best split
        for feature_index in range(X.shape[1]): #iterate over all the features
            thresholds = np.unique(X[:, feature_index]) #find the corresponding threshold to the feature
            for threshold in thresholds: #iterate over all the thresholds

                #Predict all samples in X based on feature and threshold
                predictions = np.where(X[:, feature_index] <= threshold, self.classes[0], self.classes[1]) #OBS: Only predicts one of two first classes

                #Accuracy of this prediction
                accuracy = np.mean(y == predictions)

                # Update the best split if accuracy improves
                if accuracy > best_accuracy:
                    self.feature_index = feature_index
                    self.threshold = threshold
                    best_accuracy = accuracy

    #Predict class for sample x. OBS: Only predicts one of two first classes
    def predict_instance(self, x):
        if float(x[self.feature_index]) <= float(self.threshold):
            return self.classes[0]
        else:
            return self.classes[1]

    def predict(self, X):
        #Make sure X is a DataFrame
        if isinstance(X, np.ndarray):
            X = pd.DataFrame(X)
        predictions = [] #empty list of predictions
        for _, x in X.iterrows():
            predictions.append(self.predict_instance(x)) #add predictions
        return np.array(predictions)
    

### 2.1.2 Decision Tree
To evaluate a split in a decision tree where the depth is higher than 1, it is better to evaluate a split based on information gain, rather than just accuracy. This is because a split can sort the nodes, and therefore provide information to the tree, without the accuracy being improved in that spesific split. The following decision tree will use entropy and information gain to evaluate what the best split is. 

#### Entropy
To compare the impurity before and after a split, I need a function for calculating entropy. Simply explained, entropy is a way of measuring disorder, and is used to measure the randomness in a set. The formula for entropy is: 
$$
E(S) = \sum_{i=1}^{c} - p_i \log_2(p_i)
$$

Where S is the startset, and $p_i$ is the propability of value i. To avoid logarithm of zero, $1e-9$ is added to $p_i$. This addition is minor, but will make avoid numeric instability. 

In [37]:
def entropy(y):
    p_i = y.value_counts()/y.shape[0]
    e = np.sum(-p_i * np.log2(p_i + 1e-9))
    return e

#### Information gain
To evaluate the splits, I will calculate the entropy before and anfter the split. The difference will give me a measure of the information gain of the split. If the entropy is lower (less randomness in the subsets) after the split, the information gain will be positive. I want to choose the split that provides the highest information gain. The formula for information gain is: 
$$
InformationGain= Entropy(S) - \sum_{k} \frac{|k|}{|S|}Entropy(k)
$$

Where S is the start set, and $k_i$ is the i-th subset. The fraction factor is weighting the entropy of each of the subsets based on the number of instances in the subset, compared with the total number of instances.

In [38]:
def information_gain(y, left, right):
    y_entropy = entropy(y)
    left_entropy = entropy(left)
    right_entropy = entropy(right)
    ig = y_entropy - (len(left)/len(y)*left_entropy+len(right)/len(y)*right_entropy)
    return ig

#### Functions to find Best Split

In [39]:
#Return left and right node. OBS: Only binary split
def split(X, y, feature, threshold):
    bol = X.iloc[:,feature] <= threshold
    left = y[bol]
    right = y[~bol]
    return left, right

#Find the best split based on information gain
def get_best_split(X, y):
    best_ig = -1
    best_threshold = None
    best_feature_index = None

    for feature_id in range (1, X.shape[1]):
        thresholds = np.unique(X.iloc[: ,feature_id].tolist())
        for threshold_value in thresholds:
            left, right = split(X,y,feature_id, threshold_value)
            ig = information_gain(y,left,right)
            if ig > best_ig:
                best_ig, best_threshold,best_feature_index = ig, threshold_value, feature_id
    return best_feature_index, best_threshold

#### Decision Tree class

In [40]:
class DecisionTree:    
    def __init__(self):
        self.tree = None

    def grow_tree(self, X, y, depth=0):
            
            #Stopping criteria
            if len(np.unique(y)) == 1:
                return y.mode()[0]  # Majority class in leaf node
            
            best_feature_index, best_threshold = get_best_split(X, y)
            if best_feature_index is None:
                return y.mode()[0]  # Majority class in leaf node
            bol = X.iloc[:, best_feature_index] <= best_threshold
            y_left = y[bol]
            y_right = y[~bol]
            if y_left.empty or y_right.empty:
                return y.mode()[0]  # Majority class in leaf node
            left_tree = self.grow_tree(X[bol], y_left, depth + 1)
            right_tree = self.grow_tree(X[~bol], y_right, depth + 1)
            return (best_feature_index, best_threshold, left_tree, right_tree)

    def fit(self, X, y):
        self.tree = self.grow_tree(X, y)

    def predict(self, X):
        if isinstance(X, np.ndarray):
            X = pd.DataFrame(X)
        predictions = []
        for _, x in X.iterrows():
            predictions.append(self.pred_instance(x, self.tree))
        return np.array(predictions)

    def pred_instance(self, x, tree):
        if isinstance(tree, np.int64):
            return tree  # Return the predicted class directly
        feature_index, threshold, left_branch, right_branch = tree
        if float(x[feature_index]) <= float(threshold):
            return self.pred_instance(x, left_branch)
        else:
            return self.pred_instance(x, right_branch)

### 2.2.1 Pruned Tree
To prevent the decision tree form overfitting on the training data, a pruning method can be implemented. Pruning methods can be divided into post- and pre-pruning. Post-pruning is pruning of the tree after it is fully trained. This means that the tree is first overfitted, and then simplified to correct for the overfitting. This can take a lot of computational capacity if the tree is vert deep and the data set is big. Pre-pruning methods is aiming to stop growing the tree before it is too overfitt. In this implementation there will be a pre-pruning method where max_depth will be used. Max_depth ensures that the tree does not grow deeper than wanted. Max_depth is a hyperparameter that will be further explored in section 3. I have chosen to implement a pre-pruning method to save computational complexity, and therefore saving time and resources. 

In [59]:
class PrunedTree:
    def __init__(self, max_depth):
        self.max_depth = max_depth
        self.tree = None
    
    def grow_tree(self, X, y, max_depth, depth=0):
            #Stopping criteria. Stop if leaf node, or max_depth is reached
            if len(np.unique(y)) == 1 or depth == max_depth:
                return y.mode()[0]  # Majority class in leaf node
            
            #Get the split with the highest information gain
            best_feature_index, best_threshold = get_best_split(X, y)

            #If no split provides information gain -> leaf node
            if best_feature_index is None:
                return y.mode()[0]  # Majority class in leaf node
            
            #Split the date into left and right node based on best feature and corresponding threshold
            bol = X.iloc[:, best_feature_index] <= best_threshold
            y_left = y[bol]
            y_right = y[~bol]

            #If all samples are in one node -> leaf node
            if y_left.empty or y_right.empty:
                return y.mode()[0]  # Majority class in leaf node
            
            #If not, grow three further
            left_tree = self.grow_tree(X[bol], y_left, depth + 1)
            right_tree = self.grow_tree(X[~bol], y_right, depth + 1)

            #Return splitting feature, threshold, left and right node
            return (best_feature_index, best_threshold, left_tree, right_tree)

    #The tree grows differently than 
    def fit(self, X, y, max_depth):
        self.tree = self.grow_tree(X, y, max_depth)

    #Predict class label of x
    def pred_instance(self, x, tree):
        if isinstance(tree, np.int64):
            return tree  # Return the predicted class directly
        feature_index, threshold, left_branch, right_branch = tree
        if float(x[feature_index]) <= float(threshold):
            return self.pred_instance(x, left_branch)
        else:
            return self.pred_instance(x, right_branch)
        
    #Predict class labels for matrix X
    def predict(self, X):
        if isinstance(X, np.ndarray):
            X = pd.DataFrame(X)
        predictions = []
        for _, x in X.iterrows():
            predictions.append(self.pred_instance(x, self.tree))
        return np.array(predictions)



### 2.2 Apply Methods to Data Sets
In this section the three methods will be applied to the three data sets loaded and preprocessed in section 1. To apply the methods we have to split the data sets into feature and target, as well as splitting them into training and test set. 

#### Train & Test set


In [42]:
#Last column is target label
def feature_target_split(df):
    X = df.iloc[:, 1:] 
    y = df.iloc[:, 0]
    return X, y

#Divide data set
X1, y1 = feature_target_split(wp)
X2, y2 = feature_target_split(bcp)
X3, y3 = feature_target_split(ar)


RANDOM_STATE = 1506
np.random.seed(RANDOM_STATE)

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=RANDOM_STATE)
print("WP: ", X1_train.shape,X1_test.shape)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=RANDOM_STATE)
print("BCP: ", X2_train.shape,X2_test.shape)
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, test_size=0.2, random_state=RANDOM_STATE)
print("AR: ", X3_train.shape,X3_test.shape)

WP:  (8843, 30) (2211, 30)
BCP:  (545, 10) (137, 10)
AR:  (360, 279) (91, 279)


#### Decision Stump

In [57]:
wp_stump = DecisionStump()
bcp_stump = DecisionStump()
ar_stump = DecisionStump()

def trainStump(tree, X_train, y_train, X_test, y_test):
    tree.fit(X_train.values, y_train)
    y_pred = tree.predict(X_test.values)
    acc = np.mean(y_test == y_pred)
    return acc

wp_stump_accuracy = trainStump(wp_stump, X1_train, y1_train, X1_test, y1_test)
bcp_stump_accuracy = trainStump(bcp_stump, X2_train, y2_train, X2_test, y2_test)
ar_stump_accuracy = trainStump(ar_stump, X3_train, y3_train, X3_test, y3_test)

print("Decision Stump Accuracy")
print("Webiste Phising:", wp_stump_accuracy)
print("BCP: ", bcp_stump_accuracy)
print("Arrhythmia:", ar_stump_accuracy)


Decision Stump Accuracy
Webiste Phising: 0.736318407960199
BCP:  0.0
Arrhythmia: 0.01098901098901099


#### Decision Tree

In [45]:
wp_tree = DecisionTree()
bcp_tree = DecisionTree()
ar_tree = DecisionTree()

def trainTree(tree, X_train, y_train, X_test, y_test):
    tree.fit(X_train, y_train)
    y_pred = tree.predict(X_test)
    acc = np.mean(y_test == y_pred)
    return acc

print("Decision Tree Accuracy")
wp_tree_accuracy = trainTree(wp_tree, X1_train, y1_train, X1_test, y1_test)
print("Webiste Phising:", wp_tree_accuracy)

bcp_tree_accuracy = trainTree(bcp_tree, X2_train, y2_train, X2_test, y2_test)
print("BCP: ", bcp_tree_accuracy)

ar_tree_accuracy = trainTree(ar_tree, X3_train, y3_train, X3_test, y3_test)
print("Arrhythmia:", ar_tree_accuracy)

Decision Tree Accuracy
Webiste Phising: 0.8516508367254636
BCP:  0.0072992700729927005
Arrhythmia: 0.01098901098901099


'\n#Webiste-phising\ntree1 = DecisionTree()   \ntree1.fit(X1_train, y1_train)\ny1_pred = tree1.predict(X1_test)\nacc1_tree = np.mean(y1_test == y1_pred)\nprint("Accuracy webiste-phising:", acc1_tree)\n\n#BCP\ntree2 = DecisionTree()\ntree2.fit(X2_train, y2_train)\ny2_pred = tree2.predict(X2_test)\nacc2_tree = np.mean(y2_test == y2_pred)\nprint("Accuracy bcp:", acc2_tree)\n\n#Arrhythmia\ntree3 = DecisionTree()\ntree3.fit(X3_train, y3_train)\ny3_pred = tree3.predict(X3_test)\nacc3_tree = np.mean(y3_test == y3_pred)\nprint("Accuracy: arrhythmia", acc3_tree)\n'

#### Pruned Decision Tree

In [49]:
wp_pruned = PrunedTree()
bcp_pruned = PrunedTree()
ar_pruned = PrunedTree()

def trainTree(tree, X_train, y_train, X_test, y_test):
    tree.fit(X_train, y_train, 5)
    y_pred = tree.predict(X_test)
    acc = np.mean(y_test == y_pred)
    return acc

print("Decision Tree Accuracy")
wp_pruned_accuracy = trainTree(wp_pruned, X1_train, y1_train, X1_test, y1_test)
print("Webiste Phising:", wp_pruned_accuracy)

bcp_pruned_accuracy = trainTree(bcp_pruned, X2_train, y2_train, X2_test, y2_test)
print("BCP: ", bcp_pruned_accuracy)

ar_pruned_accuracy = trainTree(ar_pruned, X3_train, y3_train, X3_test, y3_test)
print("Arrhythmia:", ar_pruned_accuracy)

Decision Tree Accuracy
Webiste Phising: 0.8516508367254636
BCP:  0.0072992700729927005
Arrhythmia: 0.01098901098901099


# 3 Choosing hyperparameter
The only hyperparameters I have used in these implementations is max_depth. I will not use a 5-fold cross-validation to evaluate the max_depth values 2, 4, 5 and 10. Evaluating the different values for the hyperparameter without the cross-validation had a running time of 119m 46.8s, and implementing a 5-fold cross-validation will make the program more complex, and therefore add time. 


In [52]:
import time
max_depth_values = [2, 4, 5, 10]

def evaluate_model(X_train, y_train, X_test, y_test, max_depth):
    start_time = time.time()
    model = PrunedTree()
    model.fit(X_train, y_train, max_depth)
    y_pred = model.predict(X_test)
    accuracy = np.mean(y_test == y_pred)
    running_time = time.time() - start_time
    return accuracy, running_time

max_depth_values = [1, 2, 5, 10]

results = []

for max_depth in max_depth_values:
    acc1, time1 = evaluate_model(X1_train, y1_train, X1_test, y1_test, max_depth)
    acc2, time2 = evaluate_model(X2_train, y2_train, X2_test, y2_test, max_depth)
    acc3, time3 = evaluate_model(X3_train, y3_train, X3_test, y3_test, max_depth)

    results.append([max_depth, acc1, time1, acc2, time2, acc3, time3])

columns = ["Max Depth", "Accuracy (Website-Phising)", "Running time (Website-Phising)",
           "Accuracy (BCP)", "Running time (BCP)",
           "Accuracy (Arrhythmia)", "Running time (Arrhythmia)"]

df = pd.DataFrame(results, columns=columns)
print(df)

   Max Depth  Accuracy (Website-Phising)  Running time (Website-Phising)  \
0          1                    0.851651                      319.815966   
1          2                    0.851651                      320.680798   
2          5                    0.851651                      318.404168   
3         10                    0.851651                      317.715161   

   Accuracy (BCP)  Running time (BCP)  Accuracy (Arrhythmia)  \
0        0.007299           35.525393               0.010989   
1        0.007299           34.919551               0.010989   
2        0.007299           34.409210               0.010989   
3        0.007299           43.015231               0.010989   

   Running time (Arrhythmia)  
0                1439.898164  
1                1440.661396  
2                1438.906347  
3                1442.833914  


When testing the different hyperparameters we can see that all the depths provide the same accuracy, which can point to the fact that the pruned tree might be implemented a wrong way. We can also see that the runningtime is the same, which also supports this theory. It is expected to see an increase in accuracy and in running time with the increase of max_depth. 

# 4 Comparing models
In this section I will compare the three methods and comment if any of the methods preforms significantly worse on each data set. To do this, I will run a p-value test.

In [58]:
from scipy import stats

# Assuming you have accuracy scores for each model
accuracy_scores = {
    'Decision stump': [wp_stump_accuracy, bcp_stump_accuracy, ar_stump_accuracy],
    'Decision Tree': [wp_tree_accuracy, bcp_tree_accuracy, bcp_tree_accuracy],
    'Pruned Tree': [wp_pruned_accuracy, bcp_pruned_accuracy, ar_pruned_accuracy]
}

# Perform t-test for each pair of models
for model1, model2 in [('Decision stump', 'Decision Tree'), ('Decision stump', 'Pruned Tree'), ('Decision Tree', 'Pruned Tree')]:
    t_statistic, p_value = stats.ttest_rel(accuracy_scores[model1], accuracy_scores[model2])
    print(f"T-test for {model1} and {model2}:")
    print(f"  - T-statistic: {t_statistic}")
    print(f"  - P-value: {p_value}")
    if p_value < 0.05:
        print("  - Significant difference detected")
    else:
        print("  - No significant difference detected")


T-test for Decision stump and Decision Tree:
  - T-statistic: -1.0440295848510426
  - P-value: 0.40607210639704483
  - No significant difference detected
T-test for Decision stump and Pruned Tree:
  - T-statistic: -1.096281081586782
  - P-value: 0.3873356149505429
  - No significant difference detected
T-test for Decision Tree and Pruned Tree:
  - T-statistic: -1.0
  - P-value: 0.42264973081037427
  - No significant difference detected
